In [1]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

In [2]:
# Postgres info to connect

connection_args = {
  "host": "twitchdata.chd4n5ul8muk.us-east-2.rds.amazonaws.com",
  "user": "postgres",
    "password":"FwwBFmleh65qYxKxDVb9",
  "port": 5432,
  "dbname": "twitchdata"
}


connection = pg.connect(**connection_args)

In [3]:
# Final query after the database collection was stopped. This data was collected over the course of the week from 6/01 through 6/07.
query = "SELECT distinct(time_logged) FROM stream_data order by time_logged desc;"

pd_sql.read_sql(query, connection)

,time_logged
0,2019-06-07 03:02:52
1,2019-06-06 22:02:54
2,2019-06-06 21:02:55
3,2019-06-06 20:02:57
4,2019-06-06 19:04:09
5,2019-06-06 18:03:02
6,2019-06-06 17:04:01
7,2019-06-06 16:02:57
8,2019-06-06 15:03:54
9,2019-06-06 14:04:18


In [4]:
query = "SELECT COUNT(DISTINCT(stream_id)) FROM stream_data;"

pd_sql.read_sql(query, connection)

,count
0,350602


In [5]:
query = "SELECT COUNT(DISTINCT(game_id)) FROM game_information;"

pd_sql.read_sql(query, connection)

,count
0,532


In [6]:
query = "SELECT COUNT(DISTINCT(game_name)) FROM game_genres;"

pd_sql.read_sql(query, connection)

,count
0,1001


In [7]:
query = "SELECT Count(*) FROM stream_data;"

pd_sql.read_sql(query, connection)

,count
0,865196


## Querying the complete dataset for EDA as the new data comes in throughout the day. This query also created the general_view which we proceed to query from for all recommender workbooks.

In [8]:
query = "SELECT Count(*) FROM general_view;"

pd_sql.read_sql(query, connection)

,count
0,2954187


In [9]:
query = "SELECT * FROM general_view LIMIT 3;"

pd_sql.read_sql(query, connection)

,stream_id,user_name,game_id,stream_type,title,viewer_count,started_at,language,time_logged,game_name,pic_url,game_genres
0,34424458080,exxtrem_tv,27546,live,World of Tanks [GER] Relax,2,2019-06-06 09:10:12,de,2019-06-06 10:03:24,World of Tanks,https://static-cdn.jtvnw.net/ttv-boxart/World%...,Simulation
1,34424458080,exxtrem_tv,27546,live,World of Tanks [GER] Relax,2,2019-06-06 09:10:12,de,2019-06-06 10:03:24,World of Tanks,https://static-cdn.jtvnw.net/ttv-boxart/World%...,Action
2,34424458080,exxtrem_tv,27546,live,World of Tanks [GER] Relax,2,2019-06-06 09:10:12,de,2019-06-06 10:03:24,World of Tanks,https://static-cdn.jtvnw.net/ttv-boxart/World%...,Simulation


In [10]:
query = '''SELECT stream_id, user_id, user_name, stream_data.game_id, stream_type, title, viewer_count, started_at, language, time_logged, game_information.game_name, pic_url, game_genres.game_genres 
FROM stream_data 
LEFT JOIN game_information ON stream_data.game_id = game_information.game_id
LEFT JOIN game_genres ON game_information.game_name = game_genres.game_name;'''

complete_data = pd_sql.read_sql(query, connection)

In [11]:
complete_data.to_csv('final_general_view_06_07_19.csv', index = False)

## Getting the data for surprise recommender in the format to be funneled into the Surprise notebook.

In [12]:
query = '''SELECT user_name, game_name, game_genres, language, started_at, viewer_count, max(viewer_count)
FROM general_view
GROUP BY user_name, game_name, game_genres, language, started_at, viewer_count
ORDER BY user_name DESC, max DESC'''

surprise_grid = pd_sql.read_sql(query, connection)

In [13]:
surprise_grid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1387492 entries, 0 to 1387491
Data columns (total 7 columns):
user_name       1387492 non-null object
game_name       1385813 non-null object
game_genres     1383564 non-null object
language        1387492 non-null object
started_at      1387492 non-null datetime64[ns]
viewer_count    1387492 non-null int64
max             1387492 non-null int64
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 74.1+ MB


In [14]:
surprise_grid.to_csv('final_game_user_grid_06_07_19.csv', index = False)


In [15]:
query = '''SELECT user_name, game_name, max(viewer_count)
FROM general_view
GROUP BY user_name, game_name
ORDER BY user_name DESC, max DESC'''

minimal_grid = pd_sql.read_sql(query, connection)

In [16]:
minimal_grid.to_csv('final_minimal_grid_06_07_19.csv', index = False)

In [17]:
surprise_grid.head()

,user_name,game_name,game_genres,language,started_at,viewer_count,max
0,龜狗,Grand Theft Auto V,Action,zh,2019-06-06 07:30:59,1735,1735
1,龜狗,Grand Theft Auto V,Open World,zh,2019-06-06 07:30:59,1735,1735
2,龜狗,Grand Theft Auto V,Driving/Racing Game,zh,2019-06-06 07:30:59,1735,1735
3,龜狗,Grand Theft Auto V,Open World,zh,2019-06-06 07:30:59,1589,1589
4,龜狗,Grand Theft Auto V,Action,zh,2019-06-06 07:30:59,1589,1589


In [18]:
minimal_grid.head()

,user_name,game_name,max
0,龜狗,Grand Theft Auto V,1735
1,龜狗,League of Legends,1058
2,龜狗,Just Chatting,467
3,龜仔_,PLAYERUNKNOWN'S BATTLEGROUNDS,23
4,龜仔_,Auto Chess,12


In [19]:
import pickle

In [20]:
pickle.dump(surprise_grid, open( "surprise.p", "wb" ) )

In [32]:
genre_dict = dict()

In [33]:
type(genre_dict)

dict

In [ ]:
#recommender_for_existing_streamers/data/surprise_data_transformation_analysis
#fix the genres for the file then update this

ValueError: Lengths must match to compare

In [129]:
d = {}

for key,value in zip(surprise_grid.game_name,surprise_grid.game_genres):
    try:
        if value not in d[key]:
            d[key].append(value)
    except KeyError:
        d[key] = [value]

In [130]:
d
#if NONE == 'Retro'

{'Grand Theft Auto V': ['Action', 'Open World', 'Driving/Racing Game'],
 'League of Legends': ['MOBA'],
 'Just Chatting': ['IRL'],
 "PLAYERUNKNOWN'S BATTLEGROUNDS": ['FPS', 'Shooter'],
 'Auto Chess': ['Card & Board Game', 'Strategy'],
 'The Sims 4': ['Simulation'],
 'Dark Souls III': ['RPG', 'Series: Souls', 'Action'],
 'osu!': ['Rhythm & Music Game'],
 'Minecraft': ['Action', 'Adventure Game', 'Open World'],
 'World of Warships': ['Action', 'Simulation'],
 'Epic Seven': ['RPG'],
 'Black Desert Online': ['MMORPG', 'Open World', 'RPG'],
 'Hearthstone': ['Card & Board Game'],
 'Music & Performing Arts': ['Creative', 'IRL'],
 'Total War: Three Kingdoms': ['Strategy'],
 'Apex Legends': ['FPS', 'Shooter'],
 'Monster Hunter World': ['Action', 'RPG'],
 "Tom Clancy's The Division 2": ['Open World', 'RPG', 'Shooter'],
 'Hollow Knight': ['Action', 'Metroidvania', 'Platformer'],
 'Super Smash Bros. Ultimate': ['Fighting', 'Platformer'],
 'Sea of Thieves': ['Action'],
 'Sekiro: Shadows Die Twice':

In [133]:
for key,value in d.items():
    if value == ['NONE']:
        d[key] = ['Retro']
    if value == [None]:
        d[key] = ['Retro']        

In [118]:
for key in d.items():
    if ['NONE'] or [None] in value:
        d[key] = ['Other']

In [134]:
d

{'Grand Theft Auto V': ['Action', 'Open World', 'Driving/Racing Game'],
 'League of Legends': ['MOBA'],
 'Just Chatting': ['IRL'],
 "PLAYERUNKNOWN'S BATTLEGROUNDS": ['FPS', 'Shooter'],
 'Auto Chess': ['Card & Board Game', 'Strategy'],
 'The Sims 4': ['Simulation'],
 'Dark Souls III': ['RPG', 'Series: Souls', 'Action'],
 'osu!': ['Rhythm & Music Game'],
 'Minecraft': ['Action', 'Adventure Game', 'Open World'],
 'World of Warships': ['Action', 'Simulation'],
 'Epic Seven': ['RPG'],
 'Black Desert Online': ['MMORPG', 'Open World', 'RPG'],
 'Hearthstone': ['Card & Board Game'],
 'Music & Performing Arts': ['Creative', 'IRL'],
 'Total War: Three Kingdoms': ['Strategy'],
 'Apex Legends': ['FPS', 'Shooter'],
 'Monster Hunter World': ['Action', 'RPG'],
 "Tom Clancy's The Division 2": ['Open World', 'RPG', 'Shooter'],
 'Hollow Knight': ['Action', 'Metroidvania', 'Platformer'],
 'Super Smash Bros. Ultimate': ['Fighting', 'Platformer'],
 'Sea of Thieves': ['Action'],
 'Sekiro: Shadows Die Twice':

In [101]:
list_key_value = [[k,v] for k,v in d.items()]

In [102]:
pickle.dump(list_key_value, open( "surprise.p", "wb" ) )

In [104]:
list_key_value

[['Grand Theft Auto V', ['Action', 'Open World', 'Driving/Racing Game']],
 ['League of Legends', ['MOBA']],
 ['Just Chatting', ['IRL']],
 ["PLAYERUNKNOWN'S BATTLEGROUNDS", ['FPS', 'Shooter']],
 ['Auto Chess', ['Card & Board Game', 'Strategy']],
 ['The Sims 4', ['Simulation']],
 ['Dark Souls III', ['RPG', 'Series: Souls', 'Action']],
 ['osu!', ['Rhythm & Music Game']],
 ['Minecraft', ['Action', 'Adventure Game', 'Open World']],
 ['World of Warships', ['Action', 'Simulation']],
 ['Epic Seven', ['RPG']],
 ['Black Desert Online', ['MMORPG', 'Open World', 'RPG']],
 ['Hearthstone', ['Card & Board Game']],
 ['Music & Performing Arts', ['Creative', 'IRL']],
 ['Total War: Three Kingdoms', ['Strategy']],
 ['Apex Legends', ['FPS', 'Shooter']],
 ['Monster Hunter World', ['Action', 'RPG']],
 ["Tom Clancy's The Division 2", ['Open World', 'RPG', 'Shooter']],
 ['Hollow Knight', ['Action', 'Metroidvania', 'Platformer']],
 ['Super Smash Bros. Ultimate', ['Fighting', 'Platformer']],
 ['Sea of Thieves', 

In [62]:
columns = ['game','genre']
x = pd.DataFrame(list_key_value, columns=columns)

In [63]:
x

,game,genre
0,Grand Theft Auto V,"[Action, Open World, Driving/Racing Game]"
1,League of Legends,[MOBA]
2,Just Chatting,[IRL]
3,PLAYERUNKNOWN'S BATTLEGROUNDS,"[FPS, Shooter]"
4,Auto Chess,"[Card & Board Game, Strategy]"
5,The Sims 4,[Simulation]
6,Dark Souls III,"[RPG, Series: Souls, Action]"
7,osu!,[Rhythm & Music Game]
8,Minecraft,"[Action, Adventure Game, Open World]"
9,World of Warships,"[Action, Simulation]"


In [68]:
for row in x:
    print(row)

game
genre
